In [1]:
# import pandas and plotly
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

## Part 1: Use Plotly Express to Create U.S. Flight Paths Map

First download two data files from online sources.

The first data file contains the airport traffic information of a specific month for 221 major airports in the U.S. 

- `iata`: IATA airport code; used to uniquely identify an airport;
- `lat` and `long`: the latitude and longitude of an airport;
- `cnt`: the number of passengers carried by a specific airline company;

In [3]:
df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_airports

,iata,airport,city,state,country,lat,long,cnt
0,ORD,Chicago O'Hare International,Chicago,IL,USA,41.979595,-87.904464,25129
1,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944,21925
2,DFW,Dallas-Fort Worth International,Dallas-Fort Worth,TX,USA,32.895951,-97.037200,20662
3,PHX,Phoenix Sky Harbor International,Phoenix,AZ,USA,33.434167,-112.008056,17290
4,DEN,Denver Intl,Denver,CO,USA,39.858408,-104.667002,13781
...,...,...,...,...,...,...,...,...
216,EAU,Chippewa Valley Regional,Eau Claire,WI,USA,44.865257,-91.485072,48
217,DBQ,Dubuque Municipal,Dubuque,IA,USA,42.402959,-90.709167,48
218,RST,Rochester International,Rochester,MN,USA,43.908826,-92.497987,37
219,UTM,Tunica Municipal Airport,Tunica,MS,USA,34.681499,-90.348816,32



#### Question 1:

Let's use `px.scatter_geo()` create a scatter plot on a map to highlight geographic dispersion of these airports, and use the marker size to represent the number of passenges

In [ ]:
fig = px.scatter_geo(# replace ... below with needed code 
                     ...,
    
                     hover_name="airport", 
                     title='American Airport Traffic in Feb. 2011',
                     opacity=0.5)


# customize the hover text
# the mapping can be found by printing the Figure object
hover_text_template = ("<b>%{hovertext}</b><br><br>"
                       "<b><i>No. of Passengers</i>: %{marker.size}</b><br>"
                       "<b><i>Latitude</i>: %{lat}</b><br><b><i>Longitude</i>: %{lon}</b>")

fig.update_traces(hovertemplate=hover_text_template)


fig.update_layout(showlegend=False,
                  geo = go.layout.Geo(scope='north america',   # geographic scope to show
                                      projection_type='azimuthal equal area',
                                      showland=True,
                                      landcolor='rgb(243, 243, 243)',
                                      countrycolor='rgb(204, 204, 204)'))


Now, download the other data file, which contains all flights flown by American Airlines. Note that each flight route is defined by an adjacent airport pair with the same `path_id`.


In [19]:
df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/justinjiajia/datafiles/main/aa_flights.csv')
df_flight_paths.head(10)

,path_id,lat,lon,airline,iata,cnt
0,0,35.040222,-106.609194,AA,ABQ,444
1,0,32.895951,-97.037200,AA,DFW,444
2,1,30.194533,-97.669872,AA,AUS,166
3,1,41.979595,-87.904464,AA,ORD,166
4,2,41.938874,-72.683228,AA,BDL,162
5,2,32.895951,-97.037200,AA,DFW,162
6,3,41.938874,-72.683228,AA,BDL,56
7,3,18.439417,-66.001833,AA,SJU,56
8,4,33.562943,-86.753550,AA,BHM,168
9,4,32.895951,-97.037200,AA,DFW,168


#### Question 2:

Use `px.line_geo()` to create a line plot on a map to highlight all these flight routes:


In [ ]:
fig_2 = px.line_geo(# replace ... below with needed code 
                    ...,
    
    
                    # line_group to indicate how data points should be grouped. 
                    # Different groups (i.e., flight routes) will be represented by different lines.
                    line_group="path_id",
                    scope='north america', projection='azimuthal equal area')


fig_2.update_traces(line=dict(color="red", width=0.5))


We can add the lines in the 2nd Figure to the 1st Figure using the following code:

In [ ]:
fig.add_traces(fig_2.data)
fig.update_layout(title_text='American Airport Traffic <br>w/ AA Flight Routes in Feb. 2011')

## Part 2: Convert the Map Visual to a Dash App

### 1st Version


#### Question 3.

Please complete the missing pieces in the code snippet below to convert the interactive plots we just created in part 1 to Dash app.

This Dash app should be able to respond to user clicks on those blue bubbles and show a bar chart on the right side to visualize how numbers of passengers from/to the corresponding airport are distributed across all its connecting airports.

<img src="https://raw.githubusercontent.com/justinjiajia/img/master/python/app_1.png" />

In [ ]:
from dash import html, dcc, Dash
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

app = Dash(__name__)

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/justinjiajia/datafiles/main/aa_flights.csv')

fig = px.scatter_geo(df_airports, lon='long', lat='lat', hover_name="airport", size='cnt', 
                     custom_data=["iata"],
                     opacity=0.5, height=650, title='American Airport Traffic in Feb. 2011')

hover_text_template = ("<b>%{hovertext}</b><br><br>"
                       "<b><i>No. of Passengers</i>: %{marker.size}</b><br>"
                       "<b><i>Latitude</i>: %{lat}</b><br><b><i>Longitude</i>: %{lon}</b>")


# customize the app's appearance
fig.update_traces(hovertemplate=hover_text_template)
fig.update_layout(showlegend = False,
                  geo = go.layout.Geo(scope='north america',   
                                      projection_type = 'azimuthal equal area',
                                      showland=True,
                                      landcolor='rgb(243, 243, 243)',
                                      countrycolor = 'rgb(204, 204, 204)'))



#### complete the missing pieces in the code below

app.layout = html.Div([html.Div([dcc.Graph(id='us-flight-paths', figure=fig, 
                                           clickData={'points': [{'customdata': ['LAX']}]})],
                                style={'width': '50%', 'display': 'inline-block', 'padding': '0 20'}),
                       
                      # complete the code below to define the right panel that presents the bar chart
                      html.Div([...], 
                                style={'display': 'inline-block', 'width': '50%', 'padding': '0 20', })
                      ])


@app.callback(# provide the missing code here to define the input and output components
              ...)                     
def update_y_timeseries(received):

    if not received: 
        raise PreventUpdate    # don't update the plot if no data is received

    received_data = received['points'][0]['customdata'][0]

    all_paths = [path for path in df_flight_paths.query(f"iata=='{received_data}'").path_id]
    df_sub = df_flight_paths.query(f"path_id in {all_paths} and not (iata == '{received_data}')").sort_values(['cnt', 'iata'], ascending=[False, True])
                                                                                                                                               
    
    # complete the code below to create the bar chart
    bar_chart = px.bar(...,
                       labels={'cnt': 'No. of Passengers', 'iata': 'IATA Airport Code'},
                       title=f"AA Flgiht Routes from/to {df_airports.loc[df_airports['iata']==received_data, 'airport'].squeeze()}<br>Ranked by No. of Passengers")


    # provide your code for the return statement
    ...


app.run(jupyter_mode="external")

### 2nd Version


#### Question 4.

Please complete the missing pieces in the code snippet below to enrich the app with vasualizations of filgihts routes. Clicking on a different marker will lead to redrawing of those flight routes as well.


<img src="https://raw.githubusercontent.com/justinjiajia/img/master/python/app_2.png" />

In [ ]:
from dash import html, dcc, Dash
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

app = Dash(__name__)

df_airports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df_flight_paths = pd.read_csv('https://raw.githubusercontent.com/justinjiajia/datafiles/main/aa_flights.csv')

fig = px.scatter_geo(df_airports, lon='long', lat='lat', hover_name="airport", size='cnt', 
                     custom_data=["iata"],
                     opacity=0.5, height=650, title='American Airport Traffic in Feb. 2011')

hover_text_template = ("<b>%{hovertext}</b><br><br>"
                       "<b><i>No. of Passengers</i>: %{marker.size}</b><br>"
                       "<b><i>Latitude</i>: %{lat}</b><br><b><i>Longitude</i>: %{lon}</b>")

fig.update_traces(hovertemplate=hover_text_template)

fig.update_layout(showlegend = False,
                  geo=go.layout.Geo(scope = 'north america',    
                                    projection_type = 'azimuthal equal area',
                                    showland=True,
                                    landcolor='rgb(243, 243, 243)',
                                    countrycolor = 'rgb(204, 204, 204)'))

# complete the code below to define the layout
app.layout = html.Div([html.Div([...],  
                                style={'width': '50%', 'display': 'inline-block', 'padding': '0 20'}),
                       html.Div([...],
                                style={'display': 'inline-block', 'width': '50%', 'padding': '0 20', })

                       ])

@app.callback(# provide the missing code here to define the input and output components
              ...)                        
def update_y_timeseries(received):

    if not received: 
        raise PreventUpdate    # don't update the plot

    received_data = received['points'][0]['customdata'][0]
    
    # data preparation that zooms in on the flight routes to plot for this round
    all_paths = [path for path in df_flight_paths.query(f"iata=='{received_data}'").path_id]
    df_sub = df_flight_paths.query(f"path_id in {all_paths}")

    
    fig.data = fig.data[:1]                 # drop all flight routes drawn in the previous round

    # provide your code to create a new set of routes and add them to the existing map
    # you can refer to Question 2 for ideas
    fig_2 = ...
    fig_2 = fig_2.update_traces(line=dict(color="red", width=0.5))
    fig.add_traces(fig_2.data)    

    
    # data used to create the bar chart
    df_sub_bar = df_sub.loc[df_flight_paths['iata'] != received_data, :].sort_values(['cnt', 'iata'], ascending=[False, True])
    
    # complete the code below to create the bar chart
    bar_chart = px.bar(...,
                       labels={'cnt': 'No. of Passengers', 'iata': 'IATA Airport Code'},
                       title=f"Flgiht Paths from/to {df_airports.loc[df_airports['iata']==received_data, 'airport'].squeeze()}<br>Ranked by No. of Passengers")

    # provide your code for the return statement
    ...



app.run(jupyter_mode="external")